In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-recommendation-dataset/Ratings.csv
/kaggle/input/book-recommendation-dataset/Users.csv
/kaggle/input/book-recommendation-dataset/Books.csv


In [3]:
ratings=pd.read_csv("/kaggle/input/book-recommendation-dataset/Ratings.csv")
ratings.head()
ratings.describe

<bound method NDFrame.describe of          User-ID         ISBN  Book-Rating
0         276725   034545104X            0
1         276726   0155061224            5
2         276727   0446520802            0
3         276729   052165615X            3
4         276729   0521795028            6
...          ...          ...          ...
1149775   276704   1563526298            9
1149776   276706   0679447156            0
1149777   276709   0515107662           10
1149778   276721   0590442449           10
1149779   276723  05162443314            8

[1149780 rows x 3 columns]>

In [4]:
users= pd.read_csv("/kaggle/input/book-recommendation-dataset/Users.csv")
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
books = pd.read_csv("/kaggle/input/book-recommendation-dataset/Books.csv")
books.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
data = pd.read_csv("/kaggle/input/book-recommendation-dataset/Books.csv", nrows=50500)
data = data[["ISBN", "Book-Title", "Book-Author", "Year-Of-Publication", "Publisher"]]
data.columns = ["ISBN", "Title", "Author", "Year", "Publisher"]

In [5]:
data = data.dropna()
print(data.isnull().sum())

ISBN         0
Title        0
Author       0
Year         0
Publisher    0
dtype: int64


In [6]:
import nltk
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [7]:
#clean data ( preprocessing)
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["cleaned_Title"] = data["Title"].apply(clean)

In [8]:
print(data.cleaned_Title.sample(10))

40461         muffin mous new hous golden super shape book
37986    last enchant  book three arthurian saga arthur...
36161                       america slept  failur prevent 
48290                     play fire sexi citi night blaze 
27727                                  spacesuitwil travel
25617                                            el perfum
8912                                            edg danger
1316                                         miracl worker
41464                                  chimney sweeper boy
38547    rand mcnalli answer atlas geographi resourc st...
Name: cleaned_Title, dtype: object


In [11]:
#dropping the duplicates from the dataset
data1 = pd.Series(data.index,index=data['cleaned_Title']).drop_duplicates()

In [13]:
#similarity using tfidf
def get_similarity(title):
    new_title = False
    feature = data["cleaned_Title"].tolist()
    if not(title in feature):
        new_title = True
        feature.append(title)
    tfidf = text.TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(feature)
    similarity = cosine_similarity(tfidf_matrix)
    del tfidf
    del tfidf_matrix
    if (new_title):
        del feature
        return similarity[len(similarity) - 1]
    else:
        del feature
        index = pd.Series(data1[title])
        print(index[0])
        return similarity[index[0]]
    

In [16]:
def book_recommendation(title):
    clean_title = clean(title)
    similarity = get_similarity(clean_title)
    similarity_scores = list(enumerate(similarity))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = filter(lambda x: x[1] > 0, similarity_scores)
    similarity_scores = list(similarity_scores)
    movieindices = [i[0] for i in similarity_scores if i[0] < len(data)]
    scores = [i[1] for i in similarity_scores if i[0] < len(data)]
    result = pd.DataFrame([data.iloc[i] for i in movieindices])
    result['score'] = scores
    result = result.values.tolist()
    print("Book Recommendation: ", title)
    
    if (len(result) > 11):
        for i in range(0,11):
            print(result[i])
    else:
        for i in range(0, len(result)):
            print(result[i])
            
    del similarity
    del similarity_scores
    del movieindices
    del scores
    del result
    
book_recommendation("summer storm")
print()
book_recommendation("Heart of the Country")

232
Book Recommendation:  summer storm
['051513290X', 'Summer of Storms', 'Judith Kelman', 2002, 'Jove Books', 'summer storm', 1.0]
['0399146741', 'Summer of Storms', 'Judith Kelman', 2001, 'Putnam Publishing Group', 'summer storm', 1.0]
['0373114087', 'Summer Storm (Harlequin Presents, No 1408)', 'Robyn Donald', 1991, 'Harlequin', 'summer storm harlequin present ', 0.7700986307398957]
['0451201906', 'Storm', 'Boris Starling', 2000, 'Signet Book', 'storm', 0.7421191394795348]
['084393672X', 'Storm', 'Norah Hess', 1994, 'Leisure Books', 'storm', 0.7421191394795348]
['0553214225', 'Summer', 'Edith Wharton', 1993, 'Bantam', 'summer', 0.6702679932819072]
['1551662809', 'Just For The Summer', 'Laura Van Wormer', 1997, 'Mira', 'summer', 0.6702679932819072]
['0552995649', 'Just for the Summer', 'Judy Astley', 1994, 'Black Swan', 'summer', 0.6702679932819072]
['0425064506', 'Summer', 'Edith Wharton', 1983, 'Berkley Publishing Group', 'summer', 0.6702679932819072]
['0380710757', "Summer of '49"